In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from datetime import date

In [2]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'class':'a-price aok-align-center'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'class':'a-offscreen'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available

def get_date():
    today = date.today()
    return today

In [3]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.com/s?k=Computer&crid=10ZLP09PI12CT&sprefix=comp%2Caps%2C424&ref=nb_sb_noss_2"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[],"scraped_date":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))
        d['scraped_date'].append(get_date())

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    
    file_name = f'amazon_data_keyword_Computer_{date.today()}.csv'
    amazon_df.to_csv(file_name, header=True, index=False)

C:\Users\fahma\AppData\Local\Temp\ipykernel_4724\2915005882.py:43: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  amazon_df['title'].replace('', np.nan, inplace=True)


In [4]:
amazon_df

,title,price,rating,reviews,availability,scraped_date
0,"Lenovo V50t Gen 2 Business Desktop Computer, I...",$529.99,4.0 out of 5 stars,18 ratings,In Stock,2024-03-19
1,"GearIT 20-Pack, Cat5e Ethernet Patch Cable 10 ...",$60.98,4.7 out of 5 stars,423 ratings,In Stock,2024-03-19
2,"HP 2022 Newest All-in-One Desktop, 21.5"" FHD D...",$442.00,4.4 out of 5 stars,555 ratings,In Stock,2024-03-19
3,Dell Optiplex 7050 SFF Desktop PC Intel i7-770...,$278.00,4.0 out of 5 stars,"1,470 ratings",,2024-03-19
4,HP Elite Desktop PC Computer Intel Core i5 3.1...,$160.47,3.9 out of 5 stars,"7,946 ratings",In Stock,2024-03-19
5,Dell Optiplex 9020 Small Form Factor Desktop w...,$239.98,4.1 out of 5 stars,"3,100 ratings",In Stock,2024-03-19
6,"Laptop Screen Extender, 14.1"" FHD 1080P IPS La...",$199.99,4.4 out of 5 stars,48 ratings,In Stock,2024-03-19
7,"SGIN 15.6 Inch Laptop, 4GB RAM 128GB SSD, Lapt...",$316.69,5.0 out of 5 stars,1 rating,In Stock,2024-03-19
8,"HP Newest All-in-One Desktop, 21.5"" FHD Displa...",$619.99,5.0 out of 5 stars,1 rating,In Stock,2024-03-19
9,"HP Elite Desktop Computer PC, 3.1 GHz, Intel C...",$196.97,3.5 out of 5 stars,401 ratings,In Stock,2024-03-19
